In [ ]:
#stuff to do:
#sigma x coupling
#make the H dimensionless and introduce proper scales
#naming of lines in graph i.e binary label
#follow trajectories

In [ ]:
!pip install qutip
from qutip import *

In [ ]:
from tqdm import tqdm

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import time
from mpl_toolkits.mplot3d import Axes3D
import math

In [ ]:
#defining operators causes them to be stored in the RAM. TO decrease RAM usage, calculate operators in situ. This will increase runtime.
def operators(N,Nlevel):
  #N = number of transmons
  #Nlevel = simulation level for bosonic operators
    Neye = []
    id = [] #tensored identiry operator
    a = [] #list of tensored destruction operators
    na = [] #list of number operators
    #NOTE: a or na go as N*(Nlevel^2). Calculating them in Situ makes this go to only Nlevel^2
    for i in  range(N):
        Neye.append(qeye(Nlevel))
    
    for i in range(N):
        temp = Neye.copy()
        temp[i] = destroy(Nlevel)
        a.append(tensor(temp))
    
    for i in range(N):
        na.append(a[i].dag()*a[i])
    
    id = tensor(Neye) 
    
    return id , a , na


In [ ]:
#id, a, na = operators(2,2)

In [ ]:
#v = tensor(basis(2,0),basis(2,1))
#expect(na,v)

In [ ]:
def hamiltonian(id,a,na,Ec,Ej,T):
    #id,a,na are the operators initialised earlier
    #Ec is a number, Ej is a list of length equal to number of transmons
    #t is a number
    N = len(Ej)
    #initialize v ,t
    v = np.zeros(N)
    t = np.zeros((N,N))
    
    for i in range(N):
        v[i] = math.sqrt(8*Ej[i]*Ec)
    
    for i in range(N):
        for j in range(N):
            t[i][j] = (T/(4*(2*Ec)**(1/float(2))))*((Ej[i]*Ej[j])**(1/float(4)))
    #NOTE: t is a symmetric matrix with non-zero diagonal elements. Hence, need to be careful while summation later
  
    
    #hamiltonian has 3 parts, linear, non-linear and interation parts
    H1 = 0
    for i in range(N):
        H1 = H1 + na[i]*v[i]
    
    H2 = 0
    for i in range(N):
        H2 = H2 + na[i]*(na[i]+1)
    H2 = -0.5*H2*Ec
    
    
    
    H3 = 0
    #NOTE: summation needs to be for i !=j parts only
    for i in range(N):
        for j in range(i+1,N):

            H3 = H3 + t[i][j]*(a[i]*a[j].dag()+a[i].dag()*a[j])
    return H1 + H2 + H3
    
    

   
    
    
    


In [ ]:
def cstates(H,na):
    #returns eigenvecs and eigenvalues of states that are computational
    #i.e have at max 1 excitation of transmons
    eigen_vals, eigen_vecs = H.eigenstates()
    c_val = []
    c_vec = []
    for i in range(len(eigen_vals)):
        c = 1
        vec = eigen_vecs[i]
        for nai in na:
            if expect(nai,vec) > 1.1:
                c = 0
                break
        if c==1 :
            c_val.append(eigen_vals[i])
            c_vec.append(eigen_vecs[i])

    return c_val,c_vec

In [ ]:
#To mark trjectories, 
#1> Overlap Cstates on Normal Graph
#2> Figureout eigenstates at T= 0 and graph their evolution (these won't be eigenstates later though)
#3> Join points as on would do by hand. (no idea how to implement it yet)

In [ ]:
#e,v = cstates(hamiltonian(id,a,na,Ec,Ej,0),na)
#print(e,v)

In [ ]:
#initial setting
N = 6 #no of transmons
Nlevel = 4 #bosonic operators to be simulated by "Nlevel" levels
tlist = np.linspace(0, 0.2, 200) # coupling strength vector

#Energy scale: Ej = 20Ghz, Ec = 250MHz, T = (5-20)MHz 
#ratio = 200/2.5/.1 
Ej = np.absolute(np.random.normal(200,50, N))
Ec = 2.5
id, a, na = operators(N, Nlevel)

In [ ]:
Ej

array([44.15799025, 47.59279566, 52.46157507, 55.40424068, 49.51091266,
       47.72373511])

In [ ]:
#def plot_points(state,):


In [ ]:
x = []
y = []

for t in tqdm(tlist):
    #print(t)
    H = hamiltonian(id,a,na,Ec,Ej,t)
    evals = H.eigenenergies()
    #print(t,evals,"\n")
    for e in evals:
        x.append(t)
        y.append(e)

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [10:04<00:00,  3.02s/it]


In [ ]:
plt.figure(figsize=(15, 15))
plt.scatter(x,y,s = 0.5)
plt.title(str(N) + " tranmons " + str(Nlevel)+ " level systems")
plt.xlabel("T value")
plt.ylabel("eigenvalues")

In [ ]:
'''
#zoom for normal figure
x2 = []
y2 = []
for i in range(len(y)):
    if y[i] > 101.5 and y[i]<103:
        x2.append(x[i])
        y2.append(y[i])


plt.figure(figsize=(15, 15))
plt.scatter(x2,y2,s = 0.5)
plt.title(str(N) + " tranmons " + str(Nlevel)+ " level systems"+ " zoomed in")
plt.xlabel("T value")
plt.ylabel("eigenvalues")
'''


'\n#zoom for normal figure\nx2 = []\ny2 = []\nfor i in range(len(y)):\n    if y[i] > 101.5 and y[i]<103:\n        x2.append(x[i])\n        y2.append(y[i])\n\n\nplt.figure(figsize=(15, 15))\nplt.scatter(x2,y2,s = 0.5)\nplt.title(str(N) + " tranmons " + str(Nlevel)+ " level systems"+ " zoomed in")\nplt.xlabel("T value")\nplt.ylabel("eigenvalues")\n'

In [ ]:
#computational states only
xc = []
yc = []

for t in tqdm(tlist):
    #print(t)
    H = hamiltonian(id,a,na,Ec,Ej,t)
    evals,evecs = cstates(H, na)
    #print(t,evals,"\n")
    for e in evals:
        xc.append(t)
        yc.append(e)

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [57:56<00:00, 17.38s/it]


In [ ]:
plt.figure(figsize=(10, 10))
plt.scatter(xc,yc,s = 0.5)
plt.title(str(N) + " tranmons " + str(Nlevel)+ " level systems"+" c states")
plt.xlabel("T value")
plt.ylabel("eigenvalues")

In [ ]:
"""
#zoomed version of computational states only
xc2 = []
yc2 = []
for i in range(len(yc)):
    if yc[i] > 30 and yc[i]<45:
        xc2.append(xc[i])
        yc2.append(yc[i])


plt.figure(figsize=(10, 10))
plt.scatter(xc2,yc2,s = 0.5)
plt.title(str(N) + " tranmons " + str(Nlevel)+ " level systems c states"+ " zoomed in")
plt.xlabel("T value")
plt.ylabel("eigenvalues")
"""

'\n#zoomed version of computational states only\nxc2 = []\nyc2 = []\nfor i in range(len(yc)):\n    if yc[i] > 30 and yc[i]<45:\n        xc2.append(xc[i])\n        yc2.append(yc[i])\n\n\nplt.figure(figsize=(10, 10))\nplt.scatter(xc2,yc2,s = 0.5)\nplt.title(str(N) + " tranmons " + str(Nlevel)+ " level systems c states"+ " zoomed in")\nplt.xlabel("T value")\nplt.ylabel("eigenvalues")\n'

In [ ]:
#to overlap plots of cstates on normal
#proly the dumbest way to do it
plt.figure(figsize=(15, 15))
plt.scatter(x,y,s = 0.5)
plt.scatter(xc,yc,s=0.5,color = 'hotpink')
plt.title(str(N) + " tranmons " + str(Nlevel)+ " level systems overlapped")
plt.xlabel("T value")
plt.ylabel("eigenvalues")

In [ ]:
maxy = max(yc)
x3 = []
y3 = []
for i in range(len(y)):
    if y[i]<maxy:
        x3.append(x[i])
        y3.append(y[i])

In [ ]:
plt.figure(figsize=(20, 20))
plt.scatter(x3,y3,s = 0.5)
plt.scatter(xc,yc,s=0.5,color = 'hotpink')
plt.title(str(N) + " tranmons " + str(Nlevel)+ " level systems overlap truncated")
plt.xlabel("T value")
plt.ylabel("eigenvalues")

In [ ]:
maxy = max(yc)
x4 = []
y4 = []
for i in range(len(y)):
    if y[i]>25 and y[i]<60:
        x4.append(x[i])
        y4.append(y[i])

        
xc4 = []
yc4 = []
for i in range(len(yc)):
    if yc[i] > 25 and yc[i]<60:
        xc4.append(xc[i])
        yc4.append(yc[i])

In [ ]:
plt.figure(figsize=(20, 20))
plt.scatter(x4,y4,s = 0.5)
plt.scatter(xc4,yc4,s=0.5,color = 'hotpink')
plt.title(str(N) + " tranmons " + str(Nlevel)+ " level systems truncated"+ " zoomed in")
plt.xlabel("T value")
plt.ylabel("eigenvalues")